<center><h2>RecVis 2022<br>Final project: Aligning Subtitles in Sign Language Videos</h2><br>


In [ ]:
# check if the notebook is correctly connected to the VM instance
!nvidia-smi

Wed Dec 28 23:51:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         384G   45G  340G  12% /
tmpfs            64M     0   64M   0% /dev
shm              26G     0   26G   0% /dev/shm
tmpfs            26G   36K   26G   1% /var/colab
/dev/sda1       390G   93G  297G  24% /opt/bin
tmpfs            26G     0   26G   0% /proc/acpi
tmpfs            26G     0   26G   0% /proc/scsi
tmpfs            26G     0   26G   0% /sys/firmware


# **Preparing the environment (libraries, models, data)**


In [ ]:
!pip install ConfigArgParse==1.2.3
!pip install webvtt_py==0.4.6
!pip install jiwer==2.2.0
!pip install nltk==3.5
!pip install beartype==0.5.1
!pip install tensorboardX==2.5.1
!pip install transformers==4.18.0

     |████████████████████████████████| 42 kB 1.4 MB/s 
  Created wheel for ConfigArgParse: filename=ConfigArgParse-1.2.3-py3-none-any.whl size=19344 sha256=484eaa605ab6151856db3cbf44241ecbe70e706cf4a46c03bb81448bfd567c2b
  Stored in directory: /root/.cache/pip/wheels/8c/56/a5/1c051b2cca57d199fc048e2516424f24263cefd3ec2059990e
Successfully built ConfigArgParse
     |████████████████████████████████| 175 kB 8.0 MB/s 
     |████████████████████████████████| 2.2 MB 76.0 MB/s 
     |████████████████████████████████| 1.4 MB 7.5 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434691 sha256=2a07d5bd4e19d1ff6cae293520ae882e5b1b921ee57655f66cd265f5483a0a35
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 260 

In [ ]:
import os
import glob 
import scipy.io
import gzip, shutil

import nltk
nltk.download('stopwords')

## Clone the original project git repo 


In [ ]:
!git clone https://github.com/hannahbull/subtitle_align.git

Cloning into 'subtitle_align'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 58 (delta 18), reused 52 (delta 14), pack-reused 0
Unpacking objects: 100% (58/58), done.


## Download the entire I3D video features (DO NOT RUN)

The model takes as input video features, which come from a sign classification model trained using RGB video frames. These features are available as part of the BOBSL data release.

In [ ]:
BOBSL_USERNAME = "bobsl-00064"
BOBSL_PASSWORD = "Eeyabei7"

!wget --recursive --no-parent --continue --wait=1 \
    --no-host-directories --cut-dirs 2 \
    --user bobsl-00064 --password Eeyabei7 \
     --trust-server-names \
  https://thor.robots.ox.ac.uk/~vgg/data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/

--2022-12-27 22:23:57--  https://thor.robots.ox.ac.uk/~vgg/data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Restricted Content"
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/index.html’

bobsl/features/i3d_     [  <=>               ] 270.28K   700KB/s    in 0.4s    

2022-12-27 22:23:58 (700 KB/s) - ‘bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/index.html’ saved [276769]

Loading robots.txt; please ignore errors.
--2022-12-27 22:23:59--  https://thor.robots.ox.ac.uk/robots.txt
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 416 R

## Only download the manually aligned video features

In [ ]:
filenames = ["subtitle_align/data/bobsl_align_train.txt",
             "subtitle_align/data/bobsl_align_val.txt",
             "subtitle_align/data/bobsl_align_test.txt"]

for fn in filenames:
    f = open(fn, "r")
    cmd = 'wget --recursive --no-parent --continue --wait=2 \
        --no-host-directories --cut-dirs 2 \
        --user bobsl-00064 --password Eeyabei7 \
         --trust-server-names '
    for l in f:
        path = 'https://thor.robots.ox.ac.uk//~vgg/data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/' + l[:-1] + '/'
        os.system(cmd + path)

In [ ]:
# unzip the features 
def gz_extract(files):
    extension = ".gz"
    for item in files: # loop through items in dir
      if item.endswith(extension): # check for ".gz" extension
          file_name = item.rsplit('.',1)[0] #get file name for file within
          with gzip.open(item,"rb") as f_in, open(file_name,"wb") as f_out:
              shutil.copyfileobj(f_in, f_out)
          os.remove(item) # delete zipped file

features_path = 'data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/'
files = [f for f in glob.glob(features_path + "**/*.gz", recursive=True)]
gz_extract(files)

Let's show an example of a video I3D features.

In [ ]:
mat = scipy.io.loadmat('data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/6144064963339075095/features.mat')
mat

{'clip_gt': array([[-1., -1., -1., ..., -1., -1., -1.]]),
 'clip_ix': array([[0., 0., 0., ..., 0., 0., 0.]]),
 'preds': array([[0.00217438, 0.00144863, 0.00327492, ..., 0.        , 0.        ,
         0.        ],
        [0.00225258, 0.00121975, 0.00310516, ..., 0.        , 0.        ,
         0.        ],
        [0.00244141, 0.00125504, 0.00309563, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.00181293, 0.00449753, 0.00019026, ..., 0.        , 0.00021017,
         0.        ],
        [0.        , 0.00557327, 0.0034523 , ..., 0.        , 0.        ,
         0.        ],
        [0.00092888, 0.00187111, 0.        , ..., 0.        , 0.00341797,
         0.        ]]),
 'video_names': array(['6144064963339075095.mp4'], dtype='<U23')}

In [ ]:
mat['clip_ix'].shape

(1, 22609)

## Download the subtitles

In [ ]:
# subtitles
!wget --recursive --no-parent --continue --wait=1 \
    --no-host-directories --cut-dirs 2 \
    --user bobsl-00064 --password Eeyabei7 \
     --trust-server-names \
  https://thor.robots.ox.ac.uk//~vgg/data/bobsl/subtitles.tar.gz

# unzip the subtitles folder
!tar -xf data/bobsl/subtitles.tar.gz --directory data/bobsl/
!rm -r data/bobsl/subtitles.tar.gz

--2022-12-28 23:41:39--  https://thor.robots.ox.ac.uk//~vgg/data/bobsl/subtitles.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Restricted Content"
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 69821179 (67M) [application/octet-stream]
Saving to: ‘data/bobsl/subtitles.tar.gz’

data/bobsl/subtitle 100%[===================>]  66.59M  18.1MB/s    in 3.7s    

2022-12-28 23:41:44 (18.1 MB/s) - ‘data/bobsl/subtitles.tar.gz’ saved [69821179/69821179]

FINISHED --2022-12-28 23:41:44--
Total wall clock time: 4.6s
Downloaded: 1 files, 67M in 3.7s (18.1 MB/s)


## Download checkpoints of the pretrained models

In [ ]:
!wget "https://drive.google.com/u/0/uc?id=1GNIm1XXRDQNFNGZVbqFcyVTZo3dFzOlD&export=download&confirm=t&uuid=61dfecd8-9063-45be-b836-cfa0ec03779f&at=ACjLJWnJbQGtddv7aGPqM6KpYb36:1672251550922" -O "inference_output.zip"
!unzip inference_output.zip
!rm inference_output.zip

/bin/bash: line 0: cd: ../models: No such file or directory
rm: cannot remove 'model_fairseq.zip': No such file or directory
--2022-12-29 00:09:38--  https://drive.google.com/u/0/uc?id=1GNIm1XXRDQNFNGZVbqFcyVTZo3dFzOlD&export=download&confirm=t&uuid=61dfecd8-9063-45be-b836-cfa0ec03779f&at=ACjLJWnJbQGtddv7aGPqM6KpYb36:1672251550922
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.100, 74.125.142.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1GNIm1XXRDQNFNGZVbqFcyVTZo3dFzOlD&export=download&confirm=t&uuid=61dfecd8-9063-45be-b836-cfa0ec03779f&at=ACjLJWnJbQGtddv7aGPqM6KpYb36:1672251550922 [following]
--2022-12-29 00:09:39--  https://drive.google.com/uc?id=1GNIm1XXRDQNFNGZVbqFcyVTZo3dFzOlD&export=download&confirm=t&uuid=61dfecd8-9063-45be-b836-cfa0ec03779f&at=ACjLJWnJbQGtddv7aGPqM6KpYb36:1672251550922
Reusing existing connection

# **Finetune the model using only the manually aligned videos**

In [ ]:
!python subtitle_align/main.py \
--features_path 'data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/' \
--gt_sub_path 'data//bobsl/subtitles/manually-aligned/' \
--pr_sub_path 'data//bobsl/subtitles/audio-aligned-heuristic-correction/' \
--gpu_id 0 \
--batch_size 64 \
--n_workers 0 \
--pr_subs_delta_bias 2.7 \
--fixed_feat_len 20 \
--jitter_location \
--jitter_abs \
--jitter_loc_quantity 2. \
--load_words False \
--load_subtitles True \
--lr 1e-6 \
--save_path 'inference_output/finetune_subtitles' \
--train_videos_txt 'subtitle_align/data/bobsl_align_train.txt' \
--val_videos_txt 'subtitle_align/data/bobsl_align_val.txt' \
--test_videos_txt 'subtitle_align/data/bobsl_align_test.txt' \
--n_epochs 100 \
--concatenate_prior True \
--min_sent_len_filter 0.5 \
--max_sent_len_filter 20 \
--shuffle_words_subs 0.5 \
--drop_words_subs 0.15 \
--resume 'checkpoints_subtitle_align/train_coarse_subtitles/checkpoints/model_0000250341.pt' \

Cuda current device  0
Loading features...
 94% 15/16 [00:02<00:00,  7.88it/s]Not found: data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/5525084584552992937/features.mat
100% 16/16 [00:02<00:00,  7.58it/s]
Adding bias to prior subtitle times  2.7
Adding bias to GT subtitle times  0
Loading subtitles associated to sentences...
100% 15/15 [00:00<00:00, 28.20it/s]
mode  train
number of samples 8183
Mean feats length samples 23.528779176341196
Standard deviation feats length samples 17.43097969766977
Min feats length samples 3
Max feats length samples 123
80\% percentile feats length 35.0
90\% percentile feats length 48.0
95\% percentile feats length 59.0
Mean texts length samples 52.14517902969571
Standard deviation texts length samples 41.357559161313986
Min texts length samples 3
Max texts length samples 292
80\% percentile texts length 80.0
90\% percentile texts length 110.0
95\% percentile texts length 136.0
Loading features...
 75% 3/4 [00:00<00:00, 11.43it/s]Not found: data/bo

In [ ]:
# download the trained model for later use
from google.colab import files
files.download('inference_output/finetune_subtitles/checkpoints/model_0000263141.pt')

Testing the trained model 

In [ ]:
!python subtitle_align/main.py \
--features_path 'data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/' \
--gt_sub_path 'data//bobsl/subtitles/manually-aligned/' \
--pr_sub_path 'data//bobsl/subtitles/audio-aligned-heuristic-correction/' \
--gpu_id 0 \
--n_workers 32 \
--batch_size 1 \
--pr_subs_delta_bias 2.7 \
--fixed_feat_len 20 \
--centre_window \
--test_only \
--save_vtt True \
--save_probs True \
--dtw_postpro True \
--test_videos_txt 'subtitle_align/data/bobsl_align_test.txt' \
--resume 'inference_output/finetune_subtitles/checkpoints/model_0000263141.pt' \

# Computed over 2642663 frames, 20338 sentences - Frame-level accuracy: 70.89 F1@0.10: 74.08 F1@0.25: 66.78 F1@0.50: 53.22

### 2.7s shift baseline
# frame_accuracy: 62.40
# f1_10: 72.77
# f1_25: 64.08
# f1_50: 44.60



Cuda current device  0
Loading features...
 97% 34/35 [00:23<00:00,  1.26it/s]Not found: data/bobsl/features/i3d_c2281_16f_m8_-15_4_d0.8_-3_22/6372027224521959778/features.mat
100% 35/35 [00:23<00:00,  1.51it/s]
Adding bias to prior subtitle times  2.7
Adding bias to GT subtitle times  0
Loading subtitles associated to sentences...
100% 34/34 [00:01<00:00, 27.67it/s]
mode  test
number of samples 20985
Mean feats length samples 24.06981177031213
Standard deviation feats length samples 19.044260457208527
Min feats length samples 0
Max feats length samples 267
80\% percentile feats length 37.0
90\% percentile feats length 49.0
95\% percentile feats length 60.0
Mean texts length samples 57.98408386943055
Standard deviation texts length samples 46.59524062694889
Min texts length samples 2
Max texts length samples 549
80\% percentile texts length 91.0
90\% percentile texts length 119.0
95\% percentile texts length 147.0
Some weights of the model checkpoint at bert-base-uncased were not used 